In [1]:
import pandas as pd
import numpy as np
import csv

In [2]:
months = ['04_april', '05_may', '06_june', '07_july', '08_august', '09_september']
periods = ['weekday_1200amto0600am.csv', 'weekday_0600amto1000am.csv', 'weekday_1000amto0330pm.csv',
        'weekday_0330pmto0430pm.csv', 'weekday_0430pmto0600pm.csv', 'weekday_0600pmto0900pm.csv',
        'weekday_0900pmto1200am.csv', 'sunday_1200amto0200am.csv', 'sunday_0200amto0200pm.csv', 
        'sunday_0200pmto0400pm.csv', 'sunday_0400pmto1200am.csv']

def clean(dataframe_):
    name0 = dataframe_.columns[0]
    name1 = dataframe_.columns[-1]
    dataframe_[name0] = dataframe_[name0].apply(lambda x: x[3:-1])
    dataframe_[name1] = dataframe_[name1].apply(lambda x: int(x[:-1]))
    return dataframe_

def linear_fit(df):
    cols = df.columns[1:]
    df['reg_april-september'] = zip(df[cols[0]], df[cols[1]], df[cols[2]], df[cols[3]], df[cols[4]], df[cols[5]])
    df['reg_april-september'] = df['reg_april-september'].apply(lambda x: np.polyfit(range(6), x, 1))
    df['reg_may-september'] = zip(df[cols[1]], df[cols[2]], df[cols[3]], df[cols[4]], df[cols[5]])
    df['reg_may-september'] = df['reg_may-september'].apply(lambda x: np.polyfit(range(5), x, 1))
    return df

data = map(lambda period: map(lambda month: clean(pd.read_csv(month+'/'+period, header=None,
                                                        names = ['CT', month])), months), periods)

In [3]:
data = map(lambda d: reduce(lambda x, y: x.merge(y, how = 'outer'), d), data)
data = map(lambda d: d.fillna(value = 0), data)

In [4]:
data[0].head(3)

,CT,04_april,05_may,06_june,07_july,08_august,09_september
0,3026600,8.0,5.0,10.0,4.0,3.0,4.0
1,3082000,121.0,123.0,120.0,127.0,99.0,120.0
2,1026500,652.0,826.0,672.0,695.0,729.0,601.0


In [5]:
data = map(linear_fit, data)

In [6]:
tocsv = map(lambda x: data[x].to_csv(periods[x]), range(len(data)))